In [25]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [27]:

load_dotenv()
KEY = os.getenv("OPENAI_API_KEY") 
llm = ChatOpenAI(openai_api_key= KEY, model_name= "gpt-3.5-turbo", temperature ="0.5")

In [28]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [29]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

# tonme: Difficulty level of the question : simple| intermediate| Hard

In [30]:
prompt_template = PromptTemplate(
    input_variables = ["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [31]:
chain = LLMChain(llm = llm, prompt = prompt_template, output_key = "quiz", verbose= True)

In [32]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [33]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [34]:
evalChain = LLMChain(llm=llm, prompt= quiz_evaluation_prompt, output_key= "review", verbose= True)

In [35]:
seqChain = SequentialChain(chains = [chain, evalChain], input_variables=["text","number","subject","tone","response_json"],
                           output_variables=["quiz", "review"], verbose = True)

In [43]:
filePath = r"E:\GPT BootCamp\OPENAI\MCQGenerator\data.txt"

with open(filePath, 'r') as file:
    TEXT = file.read()

In [44]:
print(TEXT)

Ancient Rome timeline:

Prehistory:
There is archaeological evidence of human occupation of the Rome area from at least 5,000 years, but the dense layer of much younger debris obscures Palaeolithic and Neolithic sites.[4] The evidence suggesting the city's ancient foundation is also obscured by the legend of Rome's beginning involving Romulus and Remus.

The traditional date for the founding of Rome is 21 April 753 BC, following M. Terentius Varro,[5] and the city and surrounding region of Latium has continued to be inhabited with little interruption since around that time. Excavations made in 2014 have revealed a wall built long before the city's official founding year. Archaeologists uncovered a stone wall and pieces of pottery dating to the 9th century BC and the beginning of the 8th century BC, and there is evidence of people arriving on the Palatine hill as early as the 10th century BC.

The site of Sant'Omobono Area is crucial for understanding the related processes of monumental

In [45]:
json.dumps(RESPONSE_JSON)

NUMBER=3
SUBJECT="History"
TONE="simple"

In [46]:
with get_openai_callback() as cb:
    response=seqChain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Ancient Rome timeline:

Prehistory:
There is archaeological evidence of human occupation of the Rome area from at least 5,000 years, but the dense layer of much younger debris obscures Palaeolithic and Neolithic sites.[4] The evidence suggesting the city's ancient foundation is also obscured by the legend of Rome's beginning involving Romulus and Remus.

The traditional date for the founding of Rome is 21 April 753 BC, following M. Terentius Varro,[5] and the city and surrounding region of Latium has continued to be inhabited with little interruption since around that time. Excavations made in 2014 have revealed a wall built long before the city's official founding year. Archaeologists uncovered a stone wall and pieces of pottery dating to the 9th century BC and the beginning of the 8th century BC, and there is evidence of people arriving on the Palatine hill as early as the 10th

In [39]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:749
Prompt Tokens:533
Completion Tokens:216
Total Cost:0.0012315


In [47]:
response

{'text': "Ancient Rome timeline:\n\nPrehistory:\nThere is archaeological evidence of human occupation of the Rome area from at least 5,000 years, but the dense layer of much younger debris obscures Palaeolithic and Neolithic sites.[4] The evidence suggesting the city's ancient foundation is also obscured by the legend of Rome's beginning involving Romulus and Remus.\n\nThe traditional date for the founding of Rome is 21 April 753 BC, following M. Terentius Varro,[5] and the city and surrounding region of Latium has continued to be inhabited with little interruption since around that time. Excavations made in 2014 have revealed a wall built long before the city's official founding year. Archaeologists uncovered a stone wall and pieces of pottery dating to the 9th century BC and the beginning of the 8th century BC, and there is evidence of people arriving on the Palatine hill as early as the 10th century BC.\n\nThe site of Sant'Omobono Area is crucial for understanding the related proces

In [48]:
quiz=response.get("quiz")
json.loads(quiz)

{'1': {'mcq': 'According to the legend, who were the offspring of the rape of an Alban princess by the war god Mars?',
  'options': {'a': 'Romulus and Remus',
   'b': 'Numitor and Alba Longa',
   'c': 'Aeneas and Aphrodite',
   'd': 'Titus Tatius and Sabine king'},
  'correct': 'a'},
 '2': {'mcq': 'Which hill in Rome was probably an outpost for the Sabines?',
  'options': {'a': 'Palatine Hill',
   'b': 'Capitoline Hill',
   'c': 'Quirinal Hill',
   'd': 'Aventine Hill'},
  'correct': 'c'},
 '3': {'mcq': 'Which ancient civilization deeply influenced Roman culture and some of the mythical Roman kings?',
  'options': {'a': 'Greeks',
   'b': 'Umbrians',
   'c': 'Etruscans',
   'd': 'Samnites'},
  'correct': 'c'}}